# Models visualised

In [ ]:
import os
if 'CI' in os.environ:
    os.environ['CDF_LIB'] = '/srv/conda/envs/notebook/lib'

import datetime as dt
import numpy as np
import xarray as xr
import matplotlib as mpl
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import hvplot.xarray

import eoxmagmod
from chaosmagpy.plot_utils import nio_colormap

In [ ]:
def grid(nlats=179, nlons=360, height=0):
    """A global grid of the specified size"""
    _lats = np.linspace(-89, 89, nlats)
    _lons = np.linspace(-179, 180, nlons)
    coords = np.empty((_lats.size, _lons.size, 3))
    coords[:,:,1], coords[:,:,0] = np.meshgrid(_lons, _lats)
    coords[:,:,2] = height # height above WGS84 in km
    return coords

def eval_model(
        time=dt.datetime(2020, 1, 1),
        coords=grid(),
        shc_model=eoxmagmod.data.IGRF13,
    ):
    """Evaluate a .shc model at a fixed time

    Args:
        time (datetime)
        coords (ndarray)
        shc_model (str): path to file

    Returns:
        dict: magnetic field vector components:
            https://intermagnet.github.io/faq/10.geomagnetic-comp.html
    """
    # Convert Python datetime to MJD2000
    epoch = eoxmagmod.util.datetime_to_decimal_year(time)
    mjd2000 = eoxmagmod.decimal_year_to_mjd2000(epoch)
    # Load model
    model = eoxmagmod.load_model_shc(shc_model)
    # Evaluate in North, East, Up coordinates
    # Specify coordinate systems according to:
    #   CT_GEODETIC_ABOVE_WGS84 = 0,
    #   CT_GEOCENTRIC_SPHERICAL = 1,
    #   CT_GEOCENTRIC_CARTESIAN = 2
    # https://github.com/ESA-VirES/MagneticModel/blob/staging/eoxmagmod/eoxmagmod/pymm_coord.h
    input_coordinate_system = 0
    output_coordinate_system = 0
    b_neu = model.eval(mjd2000, coords, input_coordinate_system, output_coordinate_system)
    # Inclination (I), declination (D), intensity (F)
    inc, dec, F = eoxmagmod.vincdecnorm(b_neu)
    return {"X": b_neu[:,:,0], "Y": b_neu[:,:,1], "Z": -b_neu[:,:,2],
            "I": inc, "D":dec, "F":F}

def eval_model_on_grid(model=eoxmagmod.data.IGRF13, heights=range(0, 1000, 100)):
    ds_set = []
    for height in heights:
        coords=grid(180, 360, height)
        output = eval_model(coords=coords, shc_model=model)
        _ds = xr.Dataset(
            {
                "Z": (("y", "x"), output["Z"]),
                "X": (("y", "x"), output["X"]),
                "Y": (("y", "x"), output["Y"]),
            },
            coords={"lat": (("y", "x"), coords[:,:,0]), "lon": (("y", "x"), coords[:,:,1])}
        )
        _ds = _ds.assign_coords({"height": height})
        ds_set += [_ds]
    ds = xr.concat(ds_set, dim="height")
    ds["height"].attrs = {"units": "km"}
    return ds

## Core field

In [ ]:
ds_igrf = eval_model_on_grid(
    model=eoxmagmod.data.IGRF13,
    heights=range(-3000, 3000, 500)
)
ds_igrf

In [ ]:
ds_igrf.hvplot.contourf(x="lon", y="lat", z="Z", levels=30)

## Lithospheric field

In [ ]:
ds_crust = eval_model_on_grid(
    model=eoxmagmod.data.CHAOS_STATIC_LATEST,
    heights=range(0, 1000, 100)
)
ds_crust

In [ ]:
ds_crust.hvplot.contourf(x="lon", y="lat", z="Z", levels=30, clim=(-100, 100), xlim=(-180, 180), ylim=(-90, 90))